<a href="https://colab.research.google.com/github/camilodlt/rtidy-python/blob/main/ALTERNANCE_API_CALL/La_bonne_alternance_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## API call to search effectively for an apprentice position (alternance) or companies likely to hire an apprentice in a given area

### Summary  
This script simulates the behavior of the la bonne alternance API website while also handling the ROME codes used in the query in a more implicit way (letting the user specifically choose its job's interests, hence the ROME codes passed to the API).  

### Description  
The script search for rome codes ('Répertoire Opérationnel des Métiers et des Emplois') in relation to a string search like 'data'.   

These ROME codes are then used with *la bonne alternance* API to find current job offers and also companies in the region that are likely to hire at the time (based on historical data) for these types of ROME codes.   

These calls recreate the behavior of their beta application available at : https://labonnealternance.pole-emploi.fr/ although it is not clear how they transform the job string search into romes codes. (for example, for the 'data' string search, I wasn't interested in all jobs (understood as professions or careers) provided by the pole emploi API. This script offers the possibility for the user to select the jobs that they want to keep to then be used in the la bonne alternance API call).  

Multiple rome codes can be passed to la bonne alternance API calls. The API also handles location (lat, lon), radius, insee code and sources as parameters. For this application, these parameters are fixed to the Toulouse area but can easily be changed to accommodate other needs. 

Id and secret codes are needed to use the Pole Emploi's APIs. These can be generated easily and at no charge at https://www.emploi-store-dev.fr/portail-developpeur-cms/home.html (selecting the correct API, in this case ROME API).  

## Outputs  
The script delivers a list of current apprentice positions returned by the la bonne alternance API and their respective url to go and check the role as well as a list of companies that could potentially welcome me in September for an apprentice position in the area.  

In [252]:
# LIBRARIES ------
import requests # Api requests
from requests.structures import CaseInsensitiveDict # easy dict 
import time # to sleep between requests
import pandas # To get prettier results

In [ ]:
# ID, SECRET CODE ------
"""
This secret file provides two variables :
client_id and client_secret, both provided at the url below when creating an account and requesting the use of an API. 
"""
import PARAMETERS_POLE_EMPLOI import * # Emploi store needs some tokens to access their API(es). You can get them here : 
                                      # https://www.emploi-store-dev.fr/portail-developpeur-cms/home.html
                                      # Create an account with an email and select one of the api to get acess. 
                                      # an User name and secret code are generated 
                                      # Documentation here: https://www.emploi-store-dev.fr/portail-developpeur-cms/home/catalogue-des-api/documentation-des-api/utiliser-les-api/client-credentials-grant/etape-1---generer-un-access-toke.html

In [308]:
# OAUTH HEADER REQUEST ------ 

data = CaseInsensitiveDict()
data['grant_type']='client_credentials' # mandatory 
data['client_id']= client_id # Your id
data['client_secret']= client_secret # Your secret code
data['scope']='application_%s %s %s' % (data['client_id'], 'api_romev1', 'nomenclatureRome') # Define which application are you requesting permission to. 
                                                                                            # Documentation : https://www.emploi-store-dev.fr/portail-developpeur-cms/home/catalogue-des-api/documentation-des-api/api/api-repertoire-operationnel-des/appellations.html;JSESSIONID_JAHIA=C76FD4B22752A52B518C55D052FA6B7C

In [309]:
# OAUTH2 ACCESS ------
"""
Generate oauth2 access token for the application api_romev1, nomenclatureRome. 
"""
post=requests.post(url="https://entreprise.pole-emploi.fr/connexion/oauth2/access_token",
              headers= {'Content-Type': 'application/x-www-form-urlencoded'},
              params= {'realm': '/partenaire'},
              data=data )
print("Status code : ",post.status_code) # Status of the request

# GET TOKEN ------
access_token=post.json()['access_token']

Status code :  200


In [310]:
# ROME HEADER REQUEST ------ 

headers= CaseInsensitiveDict()
headers["Accept"] = "application/json" # Mandatory 
headers["Authorization"] = f"Bearer {access_token}" # Provide the token 

In [314]:
# ROME APPELLATION API CALL ------
job_string = 'data'
response = requests.get('https://api.emploi-store.fr/partenaire/rome/v1/appellation',
                        headers=headers,
                        params= {'q': job_string}) # q parameter lets search jobs by strings
                        # Documentation of the parameters: https://www.emploi-store-dev.fr/portail-developpeur-cms/home/catalogue-des-api/documentation-des-api/api/api-repertoire-operationnel-des/appellations/lister-les-appellations.html

print("Status code : ", response.status_code) # Status of the request

# * Print job and their respective code ---
codes=[i['code'] for i in response.json()] # extract the codes from json response
libelle= [i['libelle'] for i in response.json()] # extract the libelle from json response

num= 0
print('--- Choose the jobs you are interested in. ex: 0,3,5 ---')
print('-'*50)
for i in libelle:
  print(num, i)
  num+=1
print('-'*50)

# * Give the user the choice of jobs that they want to keep ---
jobs_input=input(prompt='Choose job: ') # Users might want to keep some of the listed jobs
jobs_input=list(map(int, jobs_input.split(','))) # Convert to str to list
    # In my case, for ex, I do not want the last one: 'Délégué / Déléguée à la protection des données - Data Protection Officer' as it does not correspond to my skills. 

# * Assert validity ---
assert(max(jobs_input)<=len(codes)) # No number higher than the max number of jobs found
assert(min(jobs_input)>=0) # no negative index 

# * Retrieve codes from selected jobs ---
codes_keep=[codes[i] for i in jobs_input]


Status code :  200
--- Choose the jobs you are interested in. ex: 0,3,5 ---
--------------------------------------------------
0 Data analyst
1 Data miner
2 Data manager
3 Data scientist
4 Développeur / Développeuse Big Data
5 Délégué / Déléguée à la protection des données - Data Protection Officer
--------------------------------------------------
Choose job: 0,1,2,3,4


In [315]:
# ROME METIER API CALL ------
romes=[] # list to store rome codes
for i in codes_keep: # for every job code
  response_rome = requests.get('https://api.emploi-store.fr/partenaire/rome/v1/appellation/%s' % i, 
                        headers=headers)
  time.sleep(1) # sleep for 1 second
  if response_rome.status_code==200: # If valid request
    romes.append(response_rome.json()['metier']['code']) # get the rome code

romes=list(set(romes)) # Unique rome codes as one multiple jobs can have a single rome code.
                      # You can see this behavior here : https://www.pole-emploi.fr/employeur/vos-recrutements/le-rome-et-les-fiches-metiers.html

print('Your romes are: ',
      romes)

Your romes are:  ['M1403', 'M1805', 'M1802']


In [317]:
# LA BONNE ALTERNANCE API CALL ------

# * Parameters ---
latitude= 43.603289
longitude= 1.436622
insee= 31555
radius= 30 # In my case I am interested in Toulouse offers and companies. 
           # Change as needed. 
           # Others API can get the GPS coordinates like OSRM, MAPS ...  

# * API call ---
alternance=requests.get('https://labonnealternance.apprentissage.beta.gouv.fr/api/V1/jobs', # You can also find degrees in the city relevant to your search. See the documentation
                        params= {'romes': ','.join(romes),
                                 'latitude': latitude,
                                 'longitude': longitude, 
                                 'insee':insee,
                                 'radius': radius,
                                 'sources':'lbb,lba,offres',
                                 'caller':'contact@domaine nom_de_societe'}) # Params as per documentation 
                                 # Documentation : https://api.gouv.fr/documentation/api-la-bonne-alternance

print('Status code: ', alternance.status_code) # Status of the request


Status code:  200


In [354]:
# EXTRACT ALTERNANCE INFORMATION ------

# * Subset JSON reply ---
keep= alternance.json() # Get the JSON

job_information = ['contractDescription','description'] # Jobs features of interest 
place_information = ['city', 'fullAddress'] # place features of interest 

jobs_subset= keep['peJobs']['results'] # Subset jobs 
lba=keep['lbaCompanies']['results'] # subset companies. 
                                    # lbb source provided no information  

# * Extract information per job and company ---
num=0
dictio_jobs={} # dict store jobs
dictio_lba= {} # dict store companies

for job in jobs_subset: # iterate over jobs 
  dictio_jobs[num]= {'title':job['title'], # retrieve title 
                     'url': job['url'],
                'place':{p: job['place'][p] for p in place_information}, # place features 
                'job': {k:job['job'][k] for k in job_information} # job features 
                }  
  num+=1  

num= 0 
for i in lba: # iterate over companies
  dictio_lba[num]={'name': i['company']['name'], # Company name 
  'size': i['company']['size'], # company size
  'naf': i['nafs'][0]['label'], # naf label
  'romes': i['romes'][0]['label'] # rome domain of expertise
  } 
  num+=1

In [360]:
# CONVERT JOBS TO PANDAS DF ------
pandas.options.display.max_rows=200 # Max print size 
df_jobs=pandas.DataFrame.from_dict(dictio_jobs, orient= 'index') # dict to pandas 
df_jobs

,title,url,place,job
0,Data scientist (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
1,Apprenti(e) intelligence artificielle (h/f),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
2,Apprenti(e) en développement logiciel (h/f),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
3,Apprenti(e) en business development (h/f),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
4,ALTERNANCE - CHARGE DE PROJETS DATA SCIENTIST H/F,https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
5,Ingénieur DevOps Confirmé (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée indét...
6,Ingénieur DevOps (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée indét...
7,Développeur (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - TOULOUSE', 'fullAddress': '31 -...",{'contractDescription': 'Contrat à durée déter...
8,Développeur web (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '31 - SEILH', 'fullAddress': '31 - SE...",{'contractDescription': 'Contrat à durée déter...
9,Webmaster développeur / développeuse (H/F),https://candidat.pole-emploi.fr/offres/recherc...,"{'city': '32 - L ISLE JOURDAIN', 'fullAddress'...",{'contractDescription': 'Contrat à durée déter...


In [361]:
# * Url---
df_jobs['url']

0     https://candidat.pole-emploi.fr/offres/recherc...
1     https://candidat.pole-emploi.fr/offres/recherc...
2     https://candidat.pole-emploi.fr/offres/recherc...
3     https://candidat.pole-emploi.fr/offres/recherc...
4     https://candidat.pole-emploi.fr/offres/recherc...
5     https://candidat.pole-emploi.fr/offres/recherc...
6     https://candidat.pole-emploi.fr/offres/recherc...
7     https://candidat.pole-emploi.fr/offres/recherc...
8     https://candidat.pole-emploi.fr/offres/recherc...
9     https://candidat.pole-emploi.fr/offres/recherc...
10    https://candidat.pole-emploi.fr/offres/recherc...
Name: url, dtype: object

In [347]:
# CONVERT COMPANIES TO PANDAS DF ------
pandas.DataFrame.from_dict(dictio_lba, orient= 'index') # dict to pandas 
# These are the companies that are likely to hire an apprentice in Toulouse. 
# To my understanding these companies are flagged thanks to two algorithms predicting how likely a company is to hire an apprentice for a specific type of job

,name,size,naf,romes
0,2ICS,0 salarié,Conseil en systèmes et logiciels informatiques,Études et développement informatique
1,PLACEME,0 salarié,Conseil en systèmes et logiciels informatiques,Études et développement informatique
2,UNIVERSITE TOULOUSE 1 CAPITOLE,1 000 à 1 999 salariés,Enseignement supérieur,Études et prospectives socio-économiques
3,SIGNE,0 salarié,Conseil pour les affaires et autres conseils d...,Études et développement informatique
4,WEBINAGE,10 à 19 salariés,Edition de logiciels applicatifs,Études et développement informatique
5,INSTITUT ETUDES POLITIQUES,100 à 199 salariés,Enseignement supérieur,Études et développement informatique
6,DRYWAY,0 salarié,Conseil en systèmes et logiciels informatiques,Études et développement informatique
7,POSTELO,0 salarié,Programmation informatique,Études et développement informatique
8,PARTICULES PLUS FRANCE,6 à 9 salariés,Autres activités de télécommunication,Études et développement informatique
9,IPPON TECHNOLOGIES,0 salarié,Conseil en systèmes et logiciels informatiques,Études et développement informatique


In [ ]:
# *Credits ---
# Thanks to la bonnealternance, pole emploi and emploi-store for their work developping these APIs and their documentation. 

# links : 
# https://labonnealternance.apprentissage.beta.gouv.fr/
# https://www.pole-emploi.fr/employeur/vos-recrutements/le-rome-et-les-fiches-metiers.html
# https://api.gouv.fr/les-api/api-la-bonne-alternance
# https://api.gouv.fr/documentation/api-la-bonne-alternance